In [1]:
import os
import numpy  as np
import pandas as pd

In [2]:
os.chdir("C:\\Users\\sridhar\\Desktop\\DS2\\ML course kaggle\\5 Logit and RF")

In [3]:
data=pd.read_csv("credit_scoring_sample.csv")

In [4]:
data.head()

,SeriousDlqin2yrs,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,NumberOfTimes90DaysLate,NumberOfTime60-89DaysPastDueNotWorse,MonthlyIncome,NumberOfDependents
0,0,64,0,0.249908,0,0,8158.0,0.0
1,0,58,0,3870.000000,0,0,NaN,0.0
2,0,41,0,0.456127,0,0,6666.0,0.0
3,0,43,0,0.000190,0,0,10500.0,2.0
4,1,49,0,0.271820,0,0,400.0,0.0


In [5]:
data.shape

(45063, 8)

In [6]:
data.describe()

,SeriousDlqin2yrs,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,NumberOfTimes90DaysLate,NumberOfTime60-89DaysPastDueNotWorse,MonthlyIncome,NumberOfDependents
count,45063.000000,45063.000000,45063.000000,45063.000000,45063.000000,45063.000000,3.642000e+04,43946.000000
mean,0.222489,51.210949,0.750993,352.062688,0.570712,0.505870,6.452263e+03,0.789628
std,0.415922,14.651533,6.184521,2467.556358,6.173106,6.153609,1.263817e+04,1.130813
min,0.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,0.000000,40.000000,0.000000,0.176373,0.000000,0.000000,3.286000e+03,0.000000
50%,0.000000,51.000000,0.000000,0.373949,0.000000,0.000000,5.166000e+03,0.000000
75%,0.000000,62.000000,0.000000,0.857027,0.000000,0.000000,8.000000e+03,1.000000
max,1.000000,107.000000,98.000000,326442.000000,98.000000,98.000000,1.794060e+06,10.000000


In [7]:
data['SeriousDlqin2yrs'].value_counts()

0    35037
1    10026
Name: SeriousDlqin2yrs, dtype: int64

In [8]:
data.isna().sum()

SeriousDlqin2yrs                           0
age                                        0
NumberOfTime30-59DaysPastDueNotWorse       0
DebtRatio                                  0
NumberOfTimes90DaysLate                    0
NumberOfTime60-89DaysPastDueNotWorse       0
MonthlyIncome                           8643
NumberOfDependents                      1117
dtype: int64

In [9]:
data=data.fillna(data.median())

In [10]:
data.dtypes

SeriousDlqin2yrs                          int64
age                                       int64
NumberOfTime30-59DaysPastDueNotWorse      int64
DebtRatio                               float64
NumberOfTimes90DaysLate                   int64
NumberOfTime60-89DaysPastDueNotWorse      int64
MonthlyIncome                           float64
NumberOfDependents                      float64
dtype: object

In [11]:
floats=['MonthlyIncome', 'NumberOfDependents', 'DebtRatio']
for feature in floats:
    data[feature]=data[feature].astype(int)

 Make an interval estimate of the average age for the customers who delayed repayment at the 90% confidence level.   What is the resulting interval estimate?

In [12]:
#Generate bootstrap samples using the bootstrap method.
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [13]:
#Produce an interval estimate.
def intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [14]:
delayed_repayment=data[data['SeriousDlqin2yrs']==1]['age'].values

In [15]:
np.random.seed(0)

In [16]:
# Generate the samples using bootstrapping and calculate the mean for each of them
delayed_repayment_mean_scores = [np.mean(sample) 
                       for sample in get_bootstrap_samples(delayed_repayment, 1000)]


In [17]:
print('intervals are:',intervals(delayed_repayment_mean_scores,0.1))

intervals are: [45.71379414 46.12700479]


In [18]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import StandardScaler

In [19]:
y=data['SeriousDlqin2yrs']
x=data.drop('SeriousDlqin2yrs',axis=1)

In [20]:
train_x,test_x,train_y,test_y=train_test_split(x,y)

Perform a Grid Search with the scoring metric "roc_auc" for the parameter C. Which value of the parameter C is optimal?

In [21]:
parameters = {'C': (0.0001, 0.001, 0.01, 0.1, 1, 10)}

In [22]:
grid=GridSearchCV(LogisticRegression(random_state=5, class_weight='balanced'), parameters, cv=5)

In [23]:
grid.fit(x, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=5,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': (0.0001, 0.001, 0.01, 0.1, 1, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [24]:
grid.best_params_

{'C': 0.0001}

In [25]:
lr=LogisticRegression(C=.0001, random_state=5, class_weight='balanced')

In [26]:
lr.fit(train_x, train_y)

LogisticRegression(C=0.0001, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=5,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [27]:
predict=lr.predict(test_x)

In [28]:
roc_auc_score(predict, test_y)

0.7267901765741946

In [29]:
accuracy_score(predict, test_y)

0.8081839162080596

Can we consider the best model stable? The model is stable if the standard deviation on validation is less than 0.5%.

In [30]:
grid.grid_scores_

C:\Users\sridhar\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.81095, std: 0.00265, params: {'C': 0.0001},
 mean: 0.80565, std: 0.00182, params: {'C': 0.001},
 mean: 0.76462, std: 0.00295, params: {'C': 0.01},
 mean: 0.75454, std: 0.00337, params: {'C': 0.1},
 mean: 0.75215, std: 0.00325, params: {'C': 1},
 mean: 0.75279, std: 0.00354, params: {'C': 10}]

In [31]:
#Yes the model is stable

Feature importance is defined by the absolute value of its corresponding coefficient. First, you need to normalize all of the feature values so that it will be valid to compare them. What is the most important feature for the best logistic regression model?

In [32]:
SS=StandardScaler()

In [33]:
train_x2=SS.fit_transform(train_x)
test_x2=SS.fit_transform(test_x)

In [34]:
lr_feature_imp=LogisticRegression(C=.0001, random_state=5, class_weight='balanced')

In [35]:
lr_feature_imp.fit(train_x2, train_y)

LogisticRegression(C=0.0001, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=5,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [36]:
predict_feature_imp=lr_feature_imp.predict(test_x2)

In [37]:
lr_feature_imp.coef_

array([[-0.20586976,  0.08696543, -0.01067637,  0.06761219,  0.03489546,
        -0.03807454,  0.06794115]])

In [38]:
coeffs = np.reshape(lr_feature_imp.coef_,(-1,1))

In [39]:
pd.DataFrame(coeffs, 
             index = train_x.columns, 
             columns=['importance']).sort_values('importance',ascending=False)

,importance
NumberOfTime30-59DaysPastDueNotWorse,0.086965
NumberOfDependents,0.067941
NumberOfTimes90DaysLate,0.067612
NumberOfTime60-89DaysPastDueNotWorse,0.034895
DebtRatio,-0.010676
MonthlyIncome,-0.038075
age,-0.205870


In [40]:
# "age" is the most important feature because it has the largest absolute coefficient

In [41]:
from sklearn.ensemble import RandomForestClassifier

In [42]:
parameters2 = {'max_features': [1, 2, 4], 'min_samples_leaf': [3, 5, 7, 9], 'max_depth': [5,10,15]}

In [43]:
grid2=GridSearchCV(RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42, class_weight='balanced'), parameters2, cv=5 )

In [44]:
grid2.fit(train_x,train_y)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False, random_state=42,
            verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_features': [1, 2, 4], 'min_samples_leaf': [3, 5, 7, 9], 'max_depth': [5, 10, 15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [45]:
grid2.best_params_

{'max_depth': 15, 'max_features': 4, 'min_samples_leaf': 3}

In [46]:
rf = RandomForestClassifier(n_estimators=100,max_depth=15, max_features=4, min_samples_leaf=3, n_jobs=-1, random_state=42,  class_weight='balanced')      

In [47]:
rf.fit(train_x, train_y)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=15, max_features=4,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=3,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False, random_state=42,
            verbose=0, warm_start=False)

In [48]:
predict_rf=rf.predict(test_x)

In [49]:
roc_auc_score(predict_rf, test_y)

0.7354548173675972

How much higher is the ROC AUC of the best random forest model than that of the best logistic regression on validation?

In [50]:
0.735-0.726

0.009000000000000008

In [51]:
# auc of best random-forestmodel is 0.9%  higher than best logistic regression model

What feature has the weakest impact in the Random Forest model?

In [52]:
pd.DataFrame(rf.feature_importances_, train_x.columns,columns=['importance']).sort_values('importance',ascending=False)

,importance
NumberOfTimes90DaysLate,0.235905
NumberOfTime30-59DaysPastDueNotWorse,0.227472
MonthlyIncome,0.172035
age,0.161175
NumberOfTime60-89DaysPastDueNotWorse,0.100603
DebtRatio,0.068299
NumberOfDependents,0.034511


In [53]:
# "NumberOfDependents" feature has the weakest impact in the random forest model

What is the most significant advantage of using Logistic Regression versus Random Forest for this problem?

   1. Spent less time for model fitting;
   2. Fewer variables to iterate;
   3. Feature interpretability;
   4. Linear properties of the algorithm.

In [54]:
# All of the above properties of logistic regression are the advantages over Random forest 
# Feature interpretability is most significant advantage

In [55]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV

In [56]:
parameters3 = {'max_features': [2, 3, 4], 'max_samples': [0.5, 0.7, 0.9]}

Fit a bagging classifier with random_state=42. For the base classifiers, use 100 logistic regressors and use RandomizedSearchCV instead of GridSearchCV. It will take a lot of time to iterate over all 54 variants, so set the maximum number of iterations for RandomizedSearchCV to 20. Don't forget to set the parameters cv and random_state=1. What is the best ROC AUC you achieve?

In [57]:
search=RandomizedSearchCV(BaggingClassifier(n_estimators=100, random_state=42), parameters3, n_iter=9, random_state=1, cv=5)

In [58]:
search.fit(train_x, train_y)

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=100, n_jobs=1, oob_score=False, random_state=42,
         verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=9, n_jobs=1,
          param_distributions={'max_features': [2, 3, 4], 'max_samples': [0.5, 0.7, 0.9]},
          pre_dispatch='2*n_jobs', random_state=1, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [59]:
search.best_params_

{'max_samples': 0.5, 'max_features': 4}

In [60]:
bagging=BaggingClassifier(n_estimators=100, max_samples=0.05, max_features=4, random_state=42)

In [61]:
bagging.fit(train_x,train_y)

BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=4, max_samples=0.05,
         n_estimators=100, n_jobs=1, oob_score=False, random_state=42,
         verbose=0, warm_start=False)

In [62]:
predict_bagging=bagging.predict(test_x)

In [63]:
roc_auc_score(predict_bagging, test_y)

0.8115104136473064

Why are these values of max_features and max_samples the best?

   1. For bagging it's important to use as few features as possible;
   2. Bagging works better on small samples;
   3. Less correlation between single models;
   4. The higher the number of features, the lower the loss of information.



In [64]:
# Because Less correlation between single models